# Functions

In [1]:
import pandas as pd
import requests
from typing import Dict, List
import re
import json

def get_github_folder_files(username: str, repo_name: str, folder_path: str,
                           file_extension: str = ".csv") -> List[str]:
    """
    GitHub API를 사용해서 특정 폴더의 파일 목록을 가져오는 함수

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")
    file_extension : str
        필터링할 파일 확장자 (기본값: ".csv")

    Returns:
    --------
    List[str]
        해당 폴더의 CSV 파일명 목록
    """

    # GitHub API URL
    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{folder_path}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()

        files_data = response.json()

        # CSV 파일만 필터링
        csv_files = []
        for file_info in files_data:
            if file_info['type'] == 'file' and file_info['name'].endswith(file_extension):
                csv_files.append(file_info['name'])

        return sorted(csv_files)

    except requests.exceptions.RequestException as e:
        print(f"❌ GitHub API 요청 실패: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: {e}")
        return []


def create_variable_name_from_filename(filename: str) -> str:
    """
    파일명에서 변수명을 생성하는 함수
    temp1_governor_지역명_숫자.csv → g숫자_지역명

    Parameters:
    -----------
    filename : str
        파일명 (예: "temp1_governor_busan_3.csv", "temp1_governor_seoul_4.csv")

    Returns:
    --------
    str
        변수명 (예: "g3_busan", "g4_seoul")
    """

    # temp1_governor_지역명_숫자.csv → g숫자_지역명
    match = re.search(r'temp1_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # 매칭되지 않으면 기본적으로 파일명 정리해서 반환
    base_name = filename.replace('.csv', '').replace('temp1_', '')
    return base_name


def load_all_csvs_from_github_folder(username: str, repo_name: str, folder_path: str) -> Dict[str, pd.DataFrame]:
    """
    GitHub 폴더에서 모든 CSV 파일을 자동으로 읽어와서 DataFrame 딕셔너리로 반환

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름 (예: "korean-elections")
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")

    Returns:
    --------
    Dict[str, pd.DataFrame]
        변수명을 키로 하고 DataFrame을 값으로 하는 딕셔너리
    """

    # 1. GitHub API로 파일 목록 가져오기
    csv_files = get_github_folder_files(username, repo_name, folder_path)

    if not csv_files:
        print("❌ CSV 파일을 찾을 수 없습니다.")
        return {}

    # 2. 각 CSV 파일 읽어오기
    base_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/refs/heads/main/{folder_path}/"
    dataframes = {}

    for filename in csv_files:
        try:
            # URL 생성
            url = base_url + filename

            # 변수명 생성
            var_name = create_variable_name_from_filename(filename)

            # CSV 읽기
            df = pd.read_csv(url)

            dataframes[var_name] = df
            print(f"✅ {filename} → {var_name}")

        except Exception as e:
            print(f"❌ {filename} 읽기 실패: {e}")

    return dataframes


def create_individual_variables(dataframes: Dict[str, pd.DataFrame], globals_dict):
    """
    딕셔너리로 저장된 DataFrame들을 개별 변수로 생성

    Parameters:
    -----------
    dataframes : Dict[str, pd.DataFrame]
        load_all_csvs_from_github_folder()의 반환값
    globals_dict : dict
        변수를 저장할 전역 네임스페이스 (globals()를 전달)
    """

    for var_name, df in dataframes.items():
        globals_dict[var_name] = df
        print(f"변수 생성: {var_name}")


# # 사용 예시
# def example_usage():
#     """사용 예시"""

#     print("=== GitHub 폴더 자동 CSV 로더 ===\n")

#     print("사용법:")
#     print("""
#     username = "your-username"
#     repo_name = "korean-elections"
#     folder_path = "temp/v1_g/3rd_2002"

#     # 모든 CSV 파일을 자동으로 읽어와서 딕셔너리로 저장
#     election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

#     # 방법 1: 딕셔너리로 접근
#     busan_df = election_data['g3_busan']
#     seoul_df = election_data['g3_seoul']

#     # 방법 2: 개별 변수로 생성
#     create_individual_variables(election_data, globals())
#     # 이제 g3_busan, g3_seoul 등을 직접 사용 가능
#     """)

#     print("\n다양한 번호 지원:")
#     print("""
#     # 3회차: temp1_governor_busan_3.csv → g3_busan
#     election_3rd = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/3rd_2002")

#     # 4회차: temp1_governor_seoul_4.csv → g4_seoul
#     election_4th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/4th_2006")

#     # 5회차: temp1_governor_daegu_5.csv → g5_daegu
#     election_5th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/5th_2010")
#     """)


# if __name__ == "__main__":
#     example_usage()

# ### 간단한 사용법

# # 한 줄로 모든 CSV 읽기
# username = "your-github-username"
# repo_name = "korean-elections"
# folder_path = "temp/v1_g/3rd_2002"

# election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# # 사용하기
# busan_df = election_data['g3_busan']
# seoul_df = election_data['g3_seoul']

# # 또는 개별 변수로 생성
# create_individual_variables(election_data, globals())
# # 이제 g3_busan, g3_seoul 직접 사용 가능

In [2]:
import pandas as pd

def show_column_info(df):
    """
    데이터프레임의 열 이름과 인덱스를 확인하는 함수
    """
    print("=== 열 이름과 인덱스 ===")
    for idx, col_name in enumerate(df.columns):
        print(f"{idx}: {col_name}")
    print(f"\n총 열 개수: {len(df.columns)}")
    print(f"총 행 개수: {len(df)}")

def rename_columns_by_index(df, index_or_indices, new_name_or_names):
    """
    인덱스를 통해 열 이름을 변경하는 함수

    Parameters:
    - df: pandas DataFrame
    - index_or_indices: 단일 인덱스(int) 또는 인덱스 리스트([int, int, ...])
    - new_name_or_names: 단일 새 이름(str) 또는 새 이름 리스트([str, str, ...])

    Returns:
    - 열 이름이 변경된 DataFrame
    """
    # 단일 값을 리스트로 변환
    if isinstance(index_or_indices, int):
        indices = [index_or_indices]
    else:
        indices = index_or_indices

    if isinstance(new_name_or_names, str):
        new_names = [new_name_or_names]
    else:
        new_names = new_name_or_names

    # 길이 검증
    if len(indices) != len(new_names):
        raise ValueError(f"인덱스 개수({len(indices)})와 새 이름 개수({len(new_names)})가 일치하지 않습니다.")

    # 인덱스 유효성 검사
    for idx in indices:
        if idx < 0 or idx >= len(df.columns):
            raise ValueError(f"유효하지 않은 인덱스: {idx}. 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 열 이름 변경
    rename_dict = {}
    for idx, new_name in zip(indices, new_names):
        old_name = df.columns[idx]
        rename_dict[old_name] = new_name
        print(f"인덱스 {idx}: '{old_name}' -> '{new_name}'")

    result_df = result_df.rename(columns=rename_dict)

    print(f"\n=== 열 이름 변경 완료 ===")
    print(f"변경된 열 개수: {len(rename_dict)}")

    return result_df

def add_sum_column(df, start_idx, end_idx, new_column_name):
    """
    연속된 인덱스 범위의 열들을 합산하여 새로운 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - start_idx: 시작 인덱스
    - end_idx: 끝 인덱스 (포함)
    - new_column_name: 새로 추가할 열의 이름

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    # 유효성 검사
    if start_idx < 0 or end_idx >= len(df.columns) or start_idx > end_idx:
        raise ValueError(f"유효하지 않은 인덱스 범위: ({start_idx}, {end_idx}). 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 합산할 열 이름들 추출
    columns_to_sum = df.columns[start_idx:end_idx + 1]

    print(f"\n=== 합산할 열들 (인덱스 {start_idx}~{end_idx}) ===")
    for idx, col_name in enumerate(columns_to_sum):
        print(f"{start_idx + idx}: {col_name}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 지정된 열들을 합산하여 새로운 열 추가
    # 숫자가 아닌 값은 0으로 처리
    result_df[new_column_name] = df[columns_to_sum].select_dtypes(include=['number']).sum(axis=1, skipna=True)

    print(f"\n=== 새로운 열 추가 완료 ===")
    print(f"새로운 열 이름: {new_column_name}")
    print(f"총 열 개수: {len(result_df.columns)}")

    return result_df

def add_zero_column(df, column_name):
    """
    DataFrame 마지막에 모든 원소가 0(int)인 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - column_name: 추가할 열의 이름 (str)

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    result_df = df.copy()
    result_df[column_name] = 0
    return result_df

# # 사용 예시
# if __name__ == "__main__":
#     # CSV 파일 읽기
#     df = pd.read_csv('temp1_president_14.csv')

#     # 열 정보 확인
#     show_column_info(df)

#     # 예시 1: 단일 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 1: 단일 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df, 0, '광역시도')

#     # 예시 2: 복수 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 2: 복수 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df_renamed, [1, 2], ['시군구', '유권자수'])

#     # 예시 3: 인덱스 4~7 열들 합산 (득표수 1~4번 후보)
#     print("\n" + "="*50)
#     print("예시 3: 열 합산")
#     print("="*50)
#     result_df = add_sum_column(df_renamed, 4, 7, '상위4후보합계')

#     # 변경된 열 정보 확인
#     print("\n" + "="*50)
#     print("변경된 열 정보")
#     print("="*50)
#     show_column_info(result_df)

#     # 결과 확인 (첫 번째 행)
#     print("\n=== 첫 번째 행 결과 ===")
#     first_row = result_df.iloc[0]
#     print(f"광역시도: {first_row['광역시도']}")
#     print(f"시군구: {first_row['시군구']}")
#     print(f"유권자수: {first_row['유권자수']}")
#     print(f"득표수_1_민주자유당_김영삼: {first_row['득표수_1_민주자유당_김영삼']}")
#     print(f"득표수_2_민주당_김대중: {first_row['득표수_2_민주당_김대중']}")
#     print(f"득표수_3_통일국민당_정주영: {first_row['득표수_3_통일국민당_정주영']}")
#     print(f"득표수_4_새한국당_이종찬: {first_row['득표수_4_새한국당_이종찬']}")
#     print(f"상위4후보합계: {first_row['상위4후보합계']}")

#     # 검증: 수동 계산과 비교
#     manual_sum = (first_row['득표수_1_민주자유당_김영삼'] +
#                   first_row['득표수_2_민주당_김대중'] +
#                   first_row['득표수_3_통일국민당_정주영'] +
#                   first_row['득표수_4_새한국당_이종찬'])

#     print(f"\n수동 계산 결과: {manual_sum}")
#     print(f"함수 계산 결과: {first_row['상위4후보합계']}")
#     print(f"계산 일치: {manual_sum == first_row['상위4후보합계']}")

#     # 결과 데이터프레임 출력 (상위 5행)
#     print("\n=== 결과 데이터프레임 (상위 5행) ===")
#     print(result_df.head())

# Data (v1, 8th_2022)

In [3]:
# 한 줄로 모든 CSV 읽기
username = "sw1kwon"
repo_name = "korean-elections"
folder_path = "temp/v1_g/8th_2022"

governor_8 = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# 개별 변수로 생성
create_individual_variables(governor_8, globals())
# 이제 g8_busan, g8_seoul 직접 사용 가능

✅ temp1_governor_busan_8.csv → g8_busan
✅ temp1_governor_chungbuk_8.csv → g8_chungbuk
✅ temp1_governor_chungnam_8.csv → g8_chungnam
✅ temp1_governor_daegu_8.csv → g8_daegu
✅ temp1_governor_daejeon_8.csv → g8_daejeon
✅ temp1_governor_gangwon_8.csv → g8_gangwon
✅ temp1_governor_gwangju_8.csv → g8_gwangju
✅ temp1_governor_gyeongbuk_8.csv → g8_gyeongbuk
✅ temp1_governor_gyeonggi_8.csv → g8_gyeonggi
✅ temp1_governor_gyeongnam_8.csv → g8_gyeongnam
✅ temp1_governor_incheon_8.csv → g8_incheon
✅ temp1_governor_jeju_8.csv → g8_jeju
✅ temp1_governor_jeonbuk_8.csv → g8_jeonbuk
✅ temp1_governor_jeonnam_8.csv → g8_jeonnam
✅ temp1_governor_sejong_8.csv → g8_sejong
✅ temp1_governor_seoul_8.csv → g8_seoul
✅ temp1_governor_ulsan_8.csv → g8_ulsan
변수 생성: g8_busan
변수 생성: g8_chungbuk
변수 생성: g8_chungnam
변수 생성: g8_daegu
변수 생성: g8_daejeon
변수 생성: g8_gangwon
변수 생성: g8_gwangju
변수 생성: g8_gyeongbuk
변수 생성: g8_gyeonggi
변수 생성: g8_gyeongnam
변수 생성: g8_incheon
변수 생성: g8_jeju
변수 생성: g8_jeonbuk
변수 생성: g8_jeonnam
변수 생성: g8_

## Seoul

In [4]:
g8_seoul.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송영길,득표수_2_국민의힘_오세훈,득표수_3_정의당_권수정,득표수_4_기본소득당_신지혜,득표수_5_무소속_김광종,득표수_계,무효투표수,기권수
0,서울특별시,합계,8378339,4455161,1733183,2608277,53840,12619,9000,4416919,38242,3923178
1,서울특별시,종로구,129816,70657,28327,40145,1151,226,167,70016,641,59159
2,서울특별시,중구,112039,60323,23811,34866,696,147,125,59645,678,51716
3,서울특별시,용산구,199061,104787,34614,67579,1354,329,192,104068,719,94274
4,서울특별시,성동구,251990,139761,51996,84320,1510,350,273,138449,1312,112229


In [5]:
show_column_info(g8_seoul)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_송영길
5: 득표수_2_국민의힘_오세훈
6: 득표수_3_정의당_권수정
7: 득표수_4_기본소득당_신지혜
8: 득표수_5_무소속_김광종
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 26


### v2.1

In [6]:
g8_seoul_21 = rename_columns_by_index(g8_seoul, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_송영길' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_오세훈' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [7]:
g8_seoul_21 = add_sum_column(g8_seoul_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_정의당_권수정
7: 득표수_4_기본소득당_신지혜
8: 득표수_5_무소속_김광종

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [8]:
g8_seoul_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_권수정,득표수_4_기본소득당_신지혜,득표수_5_무소속_김광종,득표수_계,무효투표수,기권수,그외정당
0,서울특별시,합계,8378339,4455161,1733183,2608277,53840,12619,9000,4416919,38242,3923178,75459
1,서울특별시,종로구,129816,70657,28327,40145,1151,226,167,70016,641,59159,1544
2,서울특별시,중구,112039,60323,23811,34866,696,147,125,59645,678,51716,968


In [9]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_seoul_21 = g8_seoul_21[selected_columns].copy()

g8_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,서울특별시,합계,8378339,4455161,2608277,1733183,75459,4416919,38242,3923178
1,서울특별시,종로구,129816,70657,40145,28327,1544,70016,641,59159
2,서울특별시,중구,112039,60323,34866,23811,968,59645,678,51716


In [10]:
g8_seoul_21.to_csv("temp2_1_governor_seoul_8.csv", index=False, encoding="utf-8-sig")

### v2.2

In [11]:
g8_seoul_22 = rename_columns_by_index(g8_seoul, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_송영길' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_오세훈' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [12]:
g8_seoul_22 = add_sum_column(g8_seoul_22, 6, 7, '그외정당')
g8_seoul_22 = add_sum_column(g8_seoul_22, 8, 8, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_정의당_권수정
7: 득표수_4_기본소득당_신지혜

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13

=== 합산할 열들 (인덱스 8~8) ===
8: 득표수_5_무소속_김광종

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 14


In [13]:
g8_seoul_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_권수정,득표수_4_기본소득당_신지혜,득표수_5_무소속_김광종,득표수_계,무효투표수,기권수,그외정당,무소속
0,서울특별시,합계,8378339,4455161,1733183,2608277,53840,12619,9000,4416919,38242,3923178,66459,9000
1,서울특별시,종로구,129816,70657,28327,40145,1151,226,167,70016,641,59159,1377,167
2,서울특별시,중구,112039,60323,23811,34866,696,147,125,59645,678,51716,843,125


In [14]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_seoul_22 = g8_seoul_22[selected_columns].copy()

g8_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,서울특별시,합계,8378339,4455161,2608277,1733183,66459,9000,4416919,38242,3923178
1,서울특별시,종로구,129816,70657,40145,28327,1377,167,70016,641,59159
2,서울특별시,중구,112039,60323,34866,23811,843,125,59645,678,51716


In [15]:
g8_seoul_22.to_csv("temp2_2_governor_seoul_8.csv", index=False, encoding="utf-8-sig")

## Busan

In [16]:
g8_busan.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_변성완,득표수_2_국민의힘_박형준,득표수_3_정의당_김영진,득표수_계,무효투표수,기권수
0,부산광역시,합계,2916832,1432194,455901,938601,19733,1414235,17959,1484638
1,부산광역시,중구,38236,19127,5730,12826,268,18824,303,19109
2,부산광역시,서구,93426,47053,13898,31635,701,46234,819,46373
3,부산광역시,동구,80869,40832,12337,27173,562,40072,760,40037
4,부산광역시,영도구,99395,50231,17359,31130,810,49299,932,49164


In [17]:
show_column_info(g8_busan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_변성완
5: 득표수_2_국민의힘_박형준
6: 득표수_3_정의당_김영진
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 17


### v2.1

In [18]:
g8_busan_21 = rename_columns_by_index(g8_busan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_변성완' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_박형준' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [19]:
g8_busan_21 = add_sum_column(g8_busan_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_정의당_김영진

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [20]:
g8_busan_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_김영진,득표수_계,무효투표수,기권수,그외정당
0,부산광역시,합계,2916832,1432194,455901,938601,19733,1414235,17959,1484638,19733
1,부산광역시,중구,38236,19127,5730,12826,268,18824,303,19109,268
2,부산광역시,서구,93426,47053,13898,31635,701,46234,819,46373,701


In [21]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_busan_21 = g8_busan_21[selected_columns].copy()

g8_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,부산광역시,합계,2916832,1432194,938601,455901,19733,1414235,17959,1484638
1,부산광역시,중구,38236,19127,12826,5730,268,18824,303,19109
2,부산광역시,서구,93426,47053,31635,13898,701,46234,819,46373


In [22]:
g8_busan_21.to_csv("temp2_1_governor_busan_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [23]:
g8_busan_22 = rename_columns_by_index(g8_busan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_변성완' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_박형준' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [24]:
g8_busan_22 = add_sum_column(g8_busan_22, 6, 6, '그외정당')
g8_busan_22 = add_zero_column(g8_busan_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_정의당_김영진

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [25]:
g8_busan_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_김영진,득표수_계,무효투표수,기권수,그외정당,무소속
0,부산광역시,합계,2916832,1432194,455901,938601,19733,1414235,17959,1484638,19733,0
1,부산광역시,중구,38236,19127,5730,12826,268,18824,303,19109,268,0
2,부산광역시,서구,93426,47053,13898,31635,701,46234,819,46373,701,0


In [26]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_busan_22 = g8_busan_22[selected_columns].copy()

g8_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,부산광역시,합계,2916832,1432194,938601,455901,19733,0,1414235,17959,1484638
1,부산광역시,중구,38236,19127,12826,5730,268,0,18824,303,19109
2,부산광역시,서구,93426,47053,31635,13898,701,0,46234,819,46373


In [27]:
g8_busan_22.to_csv("temp2_2_governor_busan_8.csv", index=False, encoding="utf-8-sig")

## Daegu

In [28]:
g8_daegu.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_서재헌,득표수_2_국민의힘_홍준표,득표수_3_정의당_한민정,득표수_4_기본소득당_신원호,득표수_계,무효투표수,기권수
0,대구광역시,합계,2044579,883141,156429,685159,20904,7542,870034,13107,1161438
1,대구광역시,중구,68825,29536,5610,22491,746,268,29115,421,39289
2,대구광역시,동구,297073,131007,24745,100305,2851,1048,128949,2058,166066
3,대구광역시,서구,148238,66442,8993,54059,1484,623,65159,1283,81796
4,대구광역시,남구,129053,54686,8593,43386,1252,554,53785,901,74367


In [29]:
show_column_info(g8_daegu)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_서재헌
5: 득표수_2_국민의힘_홍준표
6: 득표수_3_정의당_한민정
7: 득표수_4_기본소득당_신원호
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 9


### v2.1

In [30]:
g8_daegu_21 = rename_columns_by_index(g8_daegu, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_서재헌' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_홍준표' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [31]:
g8_daegu_21 = add_sum_column(g8_daegu_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_정의당_한민정
7: 득표수_4_기본소득당_신원호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [32]:
g8_daegu_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_한민정,득표수_4_기본소득당_신원호,득표수_계,무효투표수,기권수,그외정당
0,대구광역시,합계,2044579,883141,156429,685159,20904,7542,870034,13107,1161438,28446
1,대구광역시,중구,68825,29536,5610,22491,746,268,29115,421,39289,1014
2,대구광역시,동구,297073,131007,24745,100305,2851,1048,128949,2058,166066,3899


In [33]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_daegu_21 = g8_daegu_21[selected_columns].copy()

g8_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대구광역시,합계,2044579,883141,685159,156429,28446,870034,13107,1161438
1,대구광역시,중구,68825,29536,22491,5610,1014,29115,421,39289
2,대구광역시,동구,297073,131007,100305,24745,3899,128949,2058,166066


In [34]:
g8_daegu_21.to_csv("temp2_1_governor_daegu_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [35]:
g8_daegu_22 = rename_columns_by_index(g8_daegu, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_서재헌' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_홍준표' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [36]:
g8_daegu_22 = add_sum_column(g8_daegu_22, 6, 7, '그외정당')
g8_daegu_22 = add_zero_column(g8_daegu_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_정의당_한민정
7: 득표수_4_기본소득당_신원호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [37]:
g8_daegu_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_한민정,득표수_4_기본소득당_신원호,득표수_계,무효투표수,기권수,그외정당,무소속
0,대구광역시,합계,2044579,883141,156429,685159,20904,7542,870034,13107,1161438,28446,0
1,대구광역시,중구,68825,29536,5610,22491,746,268,29115,421,39289,1014,0
2,대구광역시,동구,297073,131007,24745,100305,2851,1048,128949,2058,166066,3899,0


In [38]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_daegu_22 = g8_daegu_22[selected_columns].copy()

g8_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대구광역시,합계,2044579,883141,685159,156429,28446,0,870034,13107,1161438
1,대구광역시,중구,68825,29536,22491,5610,1014,0,29115,421,39289
2,대구광역시,동구,297073,131007,100305,24745,3899,0,128949,2058,166066


In [39]:
g8_daegu_22.to_csv("temp2_2_governor_daegu_8.csv", index=False, encoding="utf-8-sig")

## Incheon

In [40]:
g8_incheon.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_박남춘,득표수_2_국민의힘_유정복,득표수_3_정의당_이정미,득표수_4_기본소득당_김한별,득표수_계,무효투표수,기권수
0,인천광역시,합계,2534338,1240469,545885,634250,38921,6079,1225135,15334,1293869
1,인천광역시,중구,124974,57362,24425,30190,1616,294,56525,837,67612
2,인천광역시,동구,52728,28942,11592,15595,1011,207,28405,537,23786
3,인천광역시,미추홀구,358612,160328,66673,86217,4487,890,158267,2061,198284
4,인천광역시,연수구,317883,164269,63175,90120,9133,514,162942,1327,153614


In [41]:
show_column_info(g8_incheon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_박남춘
5: 득표수_2_국민의힘_유정복
6: 득표수_3_정의당_이정미
7: 득표수_4_기본소득당_김한별
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 11


### v2.1

In [42]:
g8_incheon_21 = rename_columns_by_index(g8_incheon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_박남춘' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_유정복' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [43]:
g8_incheon_21 = add_sum_column(g8_incheon_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_정의당_이정미
7: 득표수_4_기본소득당_김한별

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [44]:
g8_incheon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_이정미,득표수_4_기본소득당_김한별,득표수_계,무효투표수,기권수,그외정당
0,인천광역시,합계,2534338,1240469,545885,634250,38921,6079,1225135,15334,1293869,45000
1,인천광역시,중구,124974,57362,24425,30190,1616,294,56525,837,67612,1910
2,인천광역시,동구,52728,28942,11592,15595,1011,207,28405,537,23786,1218


In [45]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_incheon_21 = g8_incheon_21[selected_columns].copy()

g8_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,인천광역시,합계,2534338,1240469,634250,545885,45000,1225135,15334,1293869
1,인천광역시,중구,124974,57362,30190,24425,1910,56525,837,67612
2,인천광역시,동구,52728,28942,15595,11592,1218,28405,537,23786


In [46]:
g8_incheon_21.to_csv("temp2_1_governor_incheon_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [47]:
g8_incheon_22 = rename_columns_by_index(g8_incheon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_박남춘' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_유정복' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [48]:
g8_incheon_22 = add_sum_column(g8_incheon_22, 6, 7, '그외정당')
g8_incheon_22 = add_zero_column(g8_incheon_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_정의당_이정미
7: 득표수_4_기본소득당_김한별

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [49]:
g8_incheon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_이정미,득표수_4_기본소득당_김한별,득표수_계,무효투표수,기권수,그외정당,무소속
0,인천광역시,합계,2534338,1240469,545885,634250,38921,6079,1225135,15334,1293869,45000,0
1,인천광역시,중구,124974,57362,24425,30190,1616,294,56525,837,67612,1910,0
2,인천광역시,동구,52728,28942,11592,15595,1011,207,28405,537,23786,1218,0


In [50]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_incheon_22 = g8_incheon_22[selected_columns].copy()

g8_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,인천광역시,합계,2534338,1240469,634250,545885,45000,0,1225135,15334,1293869
1,인천광역시,중구,124974,57362,30190,24425,1910,0,56525,837,67612
2,인천광역시,동구,52728,28942,15595,11592,1218,0,28405,537,23786


In [51]:
g8_incheon_22.to_csv("temp2_2_governor_incheon_8.csv", index=False, encoding="utf-8-sig")

## Gwangju

In [52]:
g8_gwangju.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_강기정,득표수_2_국민의힘_주기환,득표수_3_정의당_장연주,득표수_4_기본소득당_문현철,득표수_5_진보당_김주업,득표수_계,무효투표수,기권수
0,광주광역시,합계,1206886,454516,334699,71062,21070,3344,16595,446770,7746,752370
1,광주광역시,동구,90255,38373,27775,6994,1468,299,1197,37733,640,51882
2,광주광역시,서구,246841,97941,71873,15552,4728,645,3542,96340,1601,148900
3,광주광역시,남구,180158,71496,53512,11406,2808,459,2138,70323,1173,108662
4,광주광역시,북구,363222,137865,102468,21054,6213,1037,4831,135603,2262,225357


In [53]:
show_column_info(g8_gwangju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_강기정
5: 득표수_2_국민의힘_주기환
6: 득표수_3_정의당_장연주
7: 득표수_4_기본소득당_문현철
8: 득표수_5_진보당_김주업
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 6


### v2.1

In [54]:
g8_gwangju_21 = rename_columns_by_index(g8_gwangju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_강기정' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_주기환' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [55]:
g8_gwangju_21 = add_sum_column(g8_gwangju_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_정의당_장연주
7: 득표수_4_기본소득당_문현철
8: 득표수_5_진보당_김주업

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [56]:
g8_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_장연주,득표수_4_기본소득당_문현철,득표수_5_진보당_김주업,득표수_계,무효투표수,기권수,그외정당
0,광주광역시,합계,1206886,454516,334699,71062,21070,3344,16595,446770,7746,752370,41009
1,광주광역시,동구,90255,38373,27775,6994,1468,299,1197,37733,640,51882,2964
2,광주광역시,서구,246841,97941,71873,15552,4728,645,3542,96340,1601,148900,8915


In [57]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_gwangju_21 = g8_gwangju_21[selected_columns].copy()

g8_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,광주광역시,합계,1206886,454516,71062,334699,41009,446770,7746,752370
1,광주광역시,동구,90255,38373,6994,27775,2964,37733,640,51882
2,광주광역시,서구,246841,97941,15552,71873,8915,96340,1601,148900


In [58]:
g8_gwangju_21.to_csv("temp2_1_governor_gwangju_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [59]:
g8_gwangju_22 = rename_columns_by_index(g8_gwangju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_강기정' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_주기환' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [60]:
g8_gwangju_22 = add_sum_column(g8_gwangju_22, 6, 7, '그외정당')
g8_gwangju_22 = add_zero_column(g8_gwangju_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_정의당_장연주
7: 득표수_4_기본소득당_문현철

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [61]:
g8_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_장연주,득표수_4_기본소득당_문현철,득표수_5_진보당_김주업,득표수_계,무효투표수,기권수,그외정당,무소속
0,광주광역시,합계,1206886,454516,334699,71062,21070,3344,16595,446770,7746,752370,24414,0
1,광주광역시,동구,90255,38373,27775,6994,1468,299,1197,37733,640,51882,1767,0
2,광주광역시,서구,246841,97941,71873,15552,4728,645,3542,96340,1601,148900,5373,0


In [62]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_gwangju_22 = g8_gwangju_22[selected_columns].copy()

g8_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,광주광역시,합계,1206886,454516,71062,334699,24414,0,446770,7746,752370
1,광주광역시,동구,90255,38373,6994,27775,1767,0,37733,640,51882
2,광주광역시,서구,246841,97941,15552,71873,5373,0,96340,1601,148900


In [63]:
g8_gwangju_22.to_csv("temp2_2_governor_gwangju_8.csv", index=False, encoding="utf-8-sig")

## Daejeon

In [64]:
g8_daejeon.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_허태정,득표수_2_국민의힘_이장우,득표수_계,무효투표수,기권수
0,대전광역시,합계,1233557,612639,295555,310035,605590,7049,620918
1,대전광역시,동구,191882,94041,42619,50120,92739,1302,97841
2,대전광역시,중구,199894,100343,45434,53666,99100,1243,99551
3,대전광역시,서구,399035,192546,93222,97149,190371,2175,206489
4,대전광역시,유성구,289980,149576,78142,70109,148251,1325,140404


In [65]:
show_column_info(g8_daejeon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_허태정
5: 득표수_2_국민의힘_이장우
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 6


### v2.1 (XX)

In [66]:
g8_daejeon_21 = rename_columns_by_index(g8_daejeon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_허태정' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_이장우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [67]:
g8_daejeon_21 = add_zero_column(g8_daejeon_21, '그외정당')

In [68]:
g8_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,대전광역시,합계,1233557,612639,295555,310035,605590,7049,620918,0
1,대전광역시,동구,191882,94041,42619,50120,92739,1302,97841,0
2,대전광역시,중구,199894,100343,45434,53666,99100,1243,99551,0


In [69]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_daejeon_21 = g8_daejeon_21[selected_columns].copy()

g8_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대전광역시,합계,1233557,612639,310035,295555,0,605590,7049,620918
1,대전광역시,동구,191882,94041,50120,42619,0,92739,1302,97841
2,대전광역시,중구,199894,100343,53666,45434,0,99100,1243,99551


In [70]:
g8_daejeon_21.to_csv("temp2_1_governor_daejeon_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [71]:
g8_daejeon_22 = rename_columns_by_index(g8_daejeon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_허태정' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_이장우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [72]:
g8_daejeon_22 = add_zero_column(g8_daejeon_22, '그외정당')
g8_daejeon_22 = add_zero_column(g8_daejeon_22, '무소속')

In [73]:
g8_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,대전광역시,합계,1233557,612639,295555,310035,605590,7049,620918,0,0
1,대전광역시,동구,191882,94041,42619,50120,92739,1302,97841,0,0
2,대전광역시,중구,199894,100343,45434,53666,99100,1243,99551,0,0


In [74]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_daejeon_22 = g8_daejeon_22[selected_columns].copy()

g8_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대전광역시,합계,1233557,612639,310035,295555,0,0,605590,7049,620918
1,대전광역시,동구,191882,94041,50120,42619,0,0,92739,1302,97841
2,대전광역시,중구,199894,100343,53666,45434,0,0,99100,1243,99551


In [75]:
g8_daejeon_22.to_csv("temp2_2_governor_daejeon_8.csv", index=False, encoding="utf-8-sig")

## Ulsan

In [76]:
g8_ulsan.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_송철호,득표수_2_국민의힘_김두겸,득표수_계,무효투표수,기권수
0,울산광역시,합계,941189,491866,195430,290563,485993,5873,449323
1,울산광역시,중구,181055,98391,36596,60730,97326,1065,82664
2,울산광역시,남구,267930,134765,48425,84987,133412,1353,133165
3,울산광역시,동구,127369,70833,32032,37751,69783,1050,56536
4,울산광역시,북구,175784,88502,41248,46234,87482,1020,87282


In [77]:
show_column_info(g8_ulsan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_송철호
5: 득표수_2_국민의힘_김두겸
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 6


### v2.1 (XX)

In [78]:
g8_ulsan_21 = rename_columns_by_index(g8_ulsan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_송철호' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김두겸' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [79]:
g8_ulsan_21 = add_zero_column(g8_ulsan_21, '그외정당')

In [80]:
g8_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,울산광역시,합계,941189,491866,195430,290563,485993,5873,449323,0
1,울산광역시,중구,181055,98391,36596,60730,97326,1065,82664,0
2,울산광역시,남구,267930,134765,48425,84987,133412,1353,133165,0


In [81]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_ulsan_21 = g8_ulsan_21[selected_columns].copy()

g8_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,울산광역시,합계,941189,491866,290563,195430,0,485993,5873,449323
1,울산광역시,중구,181055,98391,60730,36596,0,97326,1065,82664
2,울산광역시,남구,267930,134765,84987,48425,0,133412,1353,133165


In [82]:
g8_ulsan_21.to_csv("temp2_1_governor_ulsan_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [83]:
g8_ulsan_22 = rename_columns_by_index(g8_ulsan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_송철호' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김두겸' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [84]:
g8_ulsan_22 = add_zero_column(g8_ulsan_22, '그외정당')
g8_ulsan_22 = add_zero_column(g8_ulsan_22, '무소속')

In [85]:
g8_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,울산광역시,합계,941189,491866,195430,290563,485993,5873,449323,0,0
1,울산광역시,중구,181055,98391,36596,60730,97326,1065,82664,0,0
2,울산광역시,남구,267930,134765,48425,84987,133412,1353,133165,0,0


In [86]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_ulsan_22 = g8_ulsan_22[selected_columns].copy()

g8_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,울산광역시,합계,941189,491866,290563,195430,0,0,485993,5873,449323
1,울산광역시,중구,181055,98391,60730,36596,0,0,97326,1065,82664
2,울산광역시,남구,267930,134765,84987,48425,0,0,133412,1353,133165


In [87]:
g8_ulsan_22.to_csv("temp2_2_governor_ulsan_8.csv", index=False, encoding="utf-8-sig")

## Sejong

In [88]:
g8_sejong.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이춘희,득표수_2_국민의힘_최민호,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,292259,149751,69995,78415,148410,1341,142508


In [89]:
show_column_info(g8_sejong)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_이춘희
5: 득표수_2_국민의힘_최민호
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 1


### v2.1 (XX)

In [90]:
g8_sejong_21 = rename_columns_by_index(g8_sejong, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이춘희' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_최민호' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [91]:
g8_sejong_21 = add_zero_column(g8_sejong_21, '그외정당')

In [92]:
g8_sejong_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,세종특별자치시,합계,292259,149751,69995,78415,148410,1341,142508,0


In [93]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_sejong_21 = g8_sejong_21[selected_columns].copy()

g8_sejong_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,292259,149751,78415,69995,0,148410,1341,142508


In [94]:
g8_sejong_21.to_csv("temp2_1_governor_sejong_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [95]:
g8_sejong_22 = rename_columns_by_index(g8_sejong, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이춘희' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_최민호' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [96]:
g8_sejong_22 = add_zero_column(g8_sejong_22, '그외정당')
g8_sejong_22 = add_zero_column(g8_sejong_22, '무소속')

In [97]:
g8_sejong_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,세종특별자치시,합계,292259,149751,69995,78415,148410,1341,142508,0,0


In [98]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_sejong_22 = g8_sejong_22[selected_columns].copy()

g8_sejong_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,292259,149751,78415,69995,0,0,148410,1341,142508


In [99]:
g8_sejong_22.to_csv("temp2_2_governor_sejong_8.csv", index=False, encoding="utf-8-sig")

## Gyeonggi

In [100]:
g8_gyeonggi.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김동연,득표수_2_국민의힘_김은혜,득표수_3_정의당_황순식,득표수_4_기본소득당_서태성,득표수_5_진보당_송영주,득표수_6_무소속_강용석,득표수_계,무효투표수,기권수
0,경기도,합계,11497206,5820631,2827593,2818680,38525,9314,13939,54758,5762809,57822,5676575
1,경기도,수원시장안구,199204,106092,53333,49817,627,153,259,869,105058,1034,93112
2,경기도,수원시권선구,352407,173856,88562,80304,1090,312,406,1580,172254,1602,178551
3,경기도,수원시팔달구,164086,79283,37937,38931,488,169,176,724,78425,858,84803
4,경기도,수원시영통구,296856,160556,81109,75184,1142,212,223,1647,159517,1039,136300


In [101]:
show_column_info(g8_gyeonggi)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_김동연
5: 득표수_2_국민의힘_김은혜
6: 득표수_3_정의당_황순식
7: 득표수_4_기본소득당_서태성
8: 득표수_5_진보당_송영주
9: 득표수_6_무소속_강용석
10: 득표수_계
11: 무효투표수
12: 기권수

총 열 개수: 13
총 행 개수: 43


### v2.1

In [102]:
g8_gyeonggi_21 = rename_columns_by_index(g8_gyeonggi, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김동연' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김은혜' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [103]:
g8_gyeonggi_21 = add_sum_column(g8_gyeonggi_21, 6, 9, '그외정당')


=== 합산할 열들 (인덱스 6~9) ===
6: 득표수_3_정의당_황순식
7: 득표수_4_기본소득당_서태성
8: 득표수_5_진보당_송영주
9: 득표수_6_무소속_강용석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14


In [104]:
g8_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_황순식,득표수_4_기본소득당_서태성,득표수_5_진보당_송영주,득표수_6_무소속_강용석,득표수_계,무효투표수,기권수,그외정당
0,경기도,합계,11497206,5820631,2827593,2818680,38525,9314,13939,54758,5762809,57822,5676575,116536
1,경기도,수원시장안구,199204,106092,53333,49817,627,153,259,869,105058,1034,93112,1908
2,경기도,수원시권선구,352407,173856,88562,80304,1090,312,406,1580,172254,1602,178551,3388


In [105]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_gyeonggi_21 = g8_gyeonggi_21[selected_columns].copy()

g8_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경기도,합계,11497206,5820631,2818680,2827593,116536,5762809,57822,5676575
1,경기도,수원시장안구,199204,106092,49817,53333,1908,105058,1034,93112
2,경기도,수원시권선구,352407,173856,80304,88562,3388,172254,1602,178551


In [106]:
g8_gyeonggi_21.to_csv("temp2_1_governor_gyeonggi_8.csv", index=False, encoding="utf-8-sig")

### v2.2

In [107]:
g8_gyeonggi_22 = rename_columns_by_index(g8_gyeonggi, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김동연' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김은혜' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [108]:
g8_gyeonggi_22 = add_sum_column(g8_gyeonggi_22, 6, 8, '그외정당')
g8_gyeonggi_22 = add_sum_column(g8_gyeonggi_22, 9, 9, '무소속')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_정의당_황순식
7: 득표수_4_기본소득당_서태성
8: 득표수_5_진보당_송영주

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14

=== 합산할 열들 (인덱스 9~9) ===
9: 득표수_6_무소속_강용석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 15


In [109]:
g8_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_황순식,득표수_4_기본소득당_서태성,득표수_5_진보당_송영주,득표수_6_무소속_강용석,득표수_계,무효투표수,기권수,그외정당,무소속
0,경기도,합계,11497206,5820631,2827593,2818680,38525,9314,13939,54758,5762809,57822,5676575,61778,54758
1,경기도,수원시장안구,199204,106092,53333,49817,627,153,259,869,105058,1034,93112,1039,869
2,경기도,수원시권선구,352407,173856,88562,80304,1090,312,406,1580,172254,1602,178551,1808,1580


In [110]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_gyeonggi_22 = g8_gyeonggi_22[selected_columns].copy()

g8_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경기도,합계,11497206,5820631,2818680,2827593,61778,54758,5762809,57822,5676575
1,경기도,수원시장안구,199204,106092,49817,53333,1039,869,105058,1034,93112
2,경기도,수원시권선구,352407,173856,80304,88562,1808,1580,172254,1602,178551


In [111]:
g8_gyeonggi_22.to_csv("temp2_2_governor_gyeonggi_8.csv", index=False, encoding="utf-8-sig")

## Gangwon

In [112]:
g8_gangwon.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_이광재,득표수_2_국민의힘_김진태,득표수_계,무효투표수,기권수
0,강원도,합계,1336080,772498,347766,409461,757227,15271,563582
1,강원도,춘천시,244406,137765,63714,72126,135840,1925,106641
2,강원도,원주시,304060,157538,77841,77298,155139,2399,146522
3,강원도,강릉시,185804,101944,42113,58225,100338,1606,83860
4,강원도,동해시,76886,42047,17499,23731,41230,817,34839


In [113]:
show_column_info(g8_gangwon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_이광재
5: 득표수_2_국민의힘_김진태
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 19


### v2.1 (XX)

In [114]:
g8_gangwon_21 = rename_columns_by_index(g8_gangwon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이광재' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김진태' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [115]:
g8_gangwon_21 = add_zero_column(g8_gangwon_21, '그외정당')

In [116]:
g8_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,강원도,합계,1336080,772498,347766,409461,757227,15271,563582,0
1,강원도,춘천시,244406,137765,63714,72126,135840,1925,106641,0
2,강원도,원주시,304060,157538,77841,77298,155139,2399,146522,0


In [117]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_gangwon_21 = g8_gangwon_21[selected_columns].copy()

g8_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,강원도,합계,1336080,772498,409461,347766,0,757227,15271,563582
1,강원도,춘천시,244406,137765,72126,63714,0,135840,1925,106641
2,강원도,원주시,304060,157538,77298,77841,0,155139,2399,146522


In [118]:
g8_gangwon_21.to_csv("temp2_1_governor_gangwon_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [119]:
g8_gangwon_22 = rename_columns_by_index(g8_gangwon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_이광재' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김진태' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [120]:
g8_gangwon_22 = add_zero_column(g8_gangwon_22, '그외정당')
g8_gangwon_22 = add_zero_column(g8_gangwon_22, '무소속')

In [121]:
g8_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,강원도,합계,1336080,772498,347766,409461,757227,15271,563582,0,0
1,강원도,춘천시,244406,137765,63714,72126,135840,1925,106641,0,0
2,강원도,원주시,304060,157538,77841,77298,155139,2399,146522,0,0


In [122]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_gangwon_22 = g8_gangwon_22[selected_columns].copy()

g8_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,강원도,합계,1336080,772498,409461,347766,0,0,757227,15271,563582
1,강원도,춘천시,244406,137765,72126,63714,0,0,135840,1925,106641
2,강원도,원주시,304060,157538,77298,77841,0,0,155139,2399,146522


In [123]:
g8_gangwon_22.to_csv("temp2_2_governor_gangwon_8.csv", index=False, encoding="utf-8-sig")

## Chungbuk

In [124]:
g8_chungbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_노영민,득표수_2_국민의힘_김영환,득표수_계,무효투표수,기권수
0,충청북도,합계,1368779,692324,284166,395517,679683,12641,676455
1,충청북도,청주시상당구,163058,79612,33296,45312,78608,1004,83446
2,충청북도,청주시서원구,165453,80317,33934,45411,79345,972,85136
3,충청북도,청주시흥덕구,223618,98308,42745,54369,97114,1194,125310
4,충청북도,청주시청원구,160395,72662,32455,39209,71664,998,87733


In [125]:
show_column_info(g8_chungbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_노영민
5: 득표수_2_국민의힘_김영환
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 15


### v2.1 (XX)

In [126]:
g8_chungbuk_21 = rename_columns_by_index(g8_chungbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_노영민' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김영환' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [127]:
g8_chungbuk_21 = add_zero_column(g8_chungbuk_21, '그외정당')

In [128]:
g8_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,충청북도,합계,1368779,692324,284166,395517,679683,12641,676455,0
1,충청북도,청주시상당구,163058,79612,33296,45312,78608,1004,83446,0
2,충청북도,청주시서원구,165453,80317,33934,45411,79345,972,85136,0


In [129]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_chungbuk_21 = g8_chungbuk_21[selected_columns].copy()

g8_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청북도,합계,1368779,692324,395517,284166,0,679683,12641,676455
1,충청북도,청주시상당구,163058,79612,45312,33296,0,78608,1004,83446
2,충청북도,청주시서원구,165453,80317,45411,33934,0,79345,972,85136


In [130]:
g8_chungbuk_21.to_csv("temp2_1_governor_chungbuk_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [131]:
g8_chungbuk_22 = rename_columns_by_index(g8_chungbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_노영민' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김영환' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [132]:
g8_chungbuk_22 = add_zero_column(g8_chungbuk_22, '그외정당')
g8_chungbuk_22 = add_zero_column(g8_chungbuk_22, '무소속')

In [133]:
g8_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청북도,합계,1368779,692324,284166,395517,679683,12641,676455,0,0
1,충청북도,청주시상당구,163058,79612,33296,45312,78608,1004,83446,0,0
2,충청북도,청주시서원구,165453,80317,33934,45411,79345,972,85136,0,0


In [134]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_chungbuk_22 = g8_chungbuk_22[selected_columns].copy()

g8_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청북도,합계,1368779,692324,395517,284166,0,0,679683,12641,676455
1,충청북도,청주시상당구,163058,79612,45312,33296,0,0,78608,1004,83446
2,충청북도,청주시서원구,165453,80317,45411,33934,0,0,79345,972,85136


In [135]:
g8_chungbuk_22.to_csv("temp2_2_governor_chungbuk_8.csv", index=False, encoding="utf-8-sig")

## Chungnam

In [136]:
g8_chungnam.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양승조,득표수_2_국민의힘_김태흠,득표수_계,무효투표수,기권수
0,충청남도,합계,1803096,898369,401308,468658,869966,28403,904727
1,충청남도,천안시서북구,290313,122388,62212,57890,120102,2286,167925
2,충청남도,천안시동남구,257709,109108,51262,55733,106995,2113,148601
3,충청남도,공주시,91847,51461,21615,28250,49865,1596,40386
4,충청남도,보령시,86264,52539,18976,31729,50705,1834,33725


In [137]:
show_column_info(g8_chungnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_양승조
5: 득표수_2_국민의힘_김태흠
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 17


### v2.1 (XX)

In [138]:
g8_chungnam_21 = rename_columns_by_index(g8_chungnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_양승조' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김태흠' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [139]:
g8_chungnam_21 = add_zero_column(g8_chungnam_21, '그외정당')

In [140]:
g8_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,충청남도,합계,1803096,898369,401308,468658,869966,28403,904727,0
1,충청남도,천안시서북구,290313,122388,62212,57890,120102,2286,167925,0
2,충청남도,천안시동남구,257709,109108,51262,55733,106995,2113,148601,0


In [141]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_chungnam_21 = g8_chungnam_21[selected_columns].copy()

g8_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청남도,합계,1803096,898369,468658,401308,0,869966,28403,904727
1,충청남도,천안시서북구,290313,122388,57890,62212,0,120102,2286,167925
2,충청남도,천안시동남구,257709,109108,55733,51262,0,106995,2113,148601


In [142]:
g8_chungnam_21.to_csv("temp2_1_governor_chungnam_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [143]:
g8_chungnam_22 = rename_columns_by_index(g8_chungnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_양승조' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_김태흠' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [144]:
g8_chungnam_22 = add_zero_column(g8_chungnam_22, '그외정당')
g8_chungnam_22 = add_zero_column(g8_chungnam_22, '무소속')

In [145]:
g8_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청남도,합계,1803096,898369,401308,468658,869966,28403,904727,0,0
1,충청남도,천안시서북구,290313,122388,62212,57890,120102,2286,167925,0,0
2,충청남도,천안시동남구,257709,109108,51262,55733,106995,2113,148601,0,0


In [146]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_chungnam_22 = g8_chungnam_22[selected_columns].copy()

g8_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청남도,합계,1803096,898369,468658,401308,0,0,869966,28403,904727
1,충청남도,천안시서북구,290313,122388,57890,62212,0,0,120102,2286,167925
2,충청남도,천안시동남구,257709,109108,55733,51262,0,0,106995,2113,148601


In [147]:
g8_chungnam_22.to_csv("temp2_2_governor_chungnam_8.csv", index=False, encoding="utf-8-sig")

## Jeonbuk

In [148]:
g8_jeonbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김관영,득표수_2_국민의힘_조배숙,득표수_계,무효투표수,기권수
0,전라북도,합계,1532133,745354,591510,128828,720338,25016,786779
1,전라북도,전주시완산구,311508,125313,99377,23509,122886,2427,186195
2,전라북도,전주시덕진구,238934,96938,75741,19377,95118,1820,141996
3,전라북도,군산시,224926,87125,73109,12003,85112,2013,137801
4,전라북도,익산시,239077,107245,83134,20682,103816,3429,131832


In [149]:
show_column_info(g8_jeonbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_김관영
5: 득표수_2_국민의힘_조배숙
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 16


### v2.1 (XX)

In [150]:
g8_jeonbuk_21 = rename_columns_by_index(g8_jeonbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김관영' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_조배숙' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [151]:
g8_jeonbuk_21 = add_zero_column(g8_jeonbuk_21, '그외정당')

In [152]:
g8_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,전라북도,합계,1532133,745354,591510,128828,720338,25016,786779,0
1,전라북도,전주시완산구,311508,125313,99377,23509,122886,2427,186195,0
2,전라북도,전주시덕진구,238934,96938,75741,19377,95118,1820,141996,0


In [153]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_jeonbuk_21 = g8_jeonbuk_21[selected_columns].copy()

g8_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라북도,합계,1532133,745354,128828,591510,0,720338,25016,786779
1,전라북도,전주시완산구,311508,125313,23509,99377,0,122886,2427,186195
2,전라북도,전주시덕진구,238934,96938,19377,75741,0,95118,1820,141996


In [154]:
g8_jeonbuk_21.to_csv("temp2_1_governor_jeonbuk_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [155]:
g8_jeonbuk_22 = rename_columns_by_index(g8_jeonbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김관영' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_조배숙' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [156]:
g8_jeonbuk_22 = add_zero_column(g8_jeonbuk_22, '그외정당')
g8_jeonbuk_22 = add_zero_column(g8_jeonbuk_22, '무소속')

In [157]:
g8_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라북도,합계,1532133,745354,591510,128828,720338,25016,786779,0,0
1,전라북도,전주시완산구,311508,125313,99377,23509,122886,2427,186195,0,0
2,전라북도,전주시덕진구,238934,96938,75741,19377,95118,1820,141996,0,0


In [158]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_jeonbuk_22 = g8_jeonbuk_22[selected_columns].copy()

g8_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라북도,합계,1532133,745354,128828,591510,0,0,720338,25016,786779
1,전라북도,전주시완산구,311508,125313,23509,99377,0,0,122886,2427,186195
2,전라북도,전주시덕진구,238934,96938,19377,75741,0,0,95118,1820,141996


In [159]:
g8_jeonbuk_22.to_csv("temp2_2_governor_jeonbuk_8.csv", index=False, encoding="utf-8-sig")

## Jeonnam

In [160]:
g8_jeonnam.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_김영록,득표수_2_국민의힘_이정현,득표수_4_진보당_민점기,득표수_계,무효투표수,기권수
0,전라남도,합계,1580098,923347,672433,167020,48336,887789,35558,656751
1,전라남도,목포시,183412,96968,76408,13045,4489,93942,3026,86444
2,전라남도,여수시,236881,109135,85684,16912,4180,106776,2359,127746
3,전라남도,순천시,235432,128163,79776,40082,5472,125330,2833,107269
4,전라남도,나주시,98951,53036,39317,8638,2984,50939,2097,45915


In [161]:
show_column_info(g8_jeonnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_김영록
5: 득표수_2_국민의힘_이정현
6: 득표수_4_진보당_민점기
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 23


### v2.1

In [162]:
g8_jeonnam_21 = rename_columns_by_index(g8_jeonnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김영록' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_이정현' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [163]:
g8_jeonnam_21 = add_sum_column(g8_jeonnam_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_진보당_민점기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [164]:
g8_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_진보당_민점기,득표수_계,무효투표수,기권수,그외정당
0,전라남도,합계,1580098,923347,672433,167020,48336,887789,35558,656751,48336
1,전라남도,목포시,183412,96968,76408,13045,4489,93942,3026,86444,4489
2,전라남도,여수시,236881,109135,85684,16912,4180,106776,2359,127746,4180


In [165]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_jeonnam_21 = g8_jeonnam_21[selected_columns].copy()

g8_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라남도,합계,1580098,923347,167020,672433,48336,887789,35558,656751
1,전라남도,목포시,183412,96968,13045,76408,4489,93942,3026,86444
2,전라남도,여수시,236881,109135,16912,85684,4180,106776,2359,127746


In [166]:
g8_jeonnam_21.to_csv("temp2_1_governor_jeonnam_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [167]:
g8_jeonnam_22 = rename_columns_by_index(g8_jeonnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_김영록' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_이정현' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [168]:
g8_jeonnam_22 = add_sum_column(g8_jeonnam_22, 6, 6, '그외정당')
g8_jeonnam_22 = add_zero_column(g8_jeonnam_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_진보당_민점기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [169]:
g8_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_진보당_민점기,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라남도,합계,1580098,923347,672433,167020,48336,887789,35558,656751,48336,0
1,전라남도,목포시,183412,96968,76408,13045,4489,93942,3026,86444,4489,0
2,전라남도,여수시,236881,109135,85684,16912,4180,106776,2359,127746,4180,0


In [170]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_jeonnam_22 = g8_jeonnam_22[selected_columns].copy()

g8_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라남도,합계,1580098,923347,167020,672433,48336,0,887789,35558,656751
1,전라남도,목포시,183412,96968,13045,76408,4489,0,93942,3026,86444
2,전라남도,여수시,236881,109135,16912,85684,4180,0,106776,2359,127746


In [171]:
g8_jeonnam_22.to_csv("temp2_2_governor_jeonnam_8.csv", index=False, encoding="utf-8-sig")

## Gyeongbuk

In [172]:
g8_gyeongbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_임미애,득표수_2_국민의힘_이철우,득표수_계,무효투표수,기권수
0,경상북도,합계,2268707,1194595,255775,904675,1160450,34145,1074112
1,경상북도,포항시북구,230960,105786,23150,80784,103934,1852,125174
2,경상북도,포항시남구,196727,90434,20195,68137,88332,2102,106293
3,경상북도,울릉군,8339,6795,1217,5270,6487,308,1544
4,경상북도,경주시,220490,109647,22875,83997,106872,2775,110843


In [173]:
show_column_info(g8_gyeongbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_임미애
5: 득표수_2_국민의힘_이철우
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 25


### v2.1 (XX)

In [174]:
g8_gyeongbuk_21 = rename_columns_by_index(g8_gyeongbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_임미애' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_이철우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [175]:
g8_gyeongbuk_21 = add_zero_column(g8_gyeongbuk_21, '그외정당')

In [176]:
g8_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,경상북도,합계,2268707,1194595,255775,904675,1160450,34145,1074112,0
1,경상북도,포항시북구,230960,105786,23150,80784,103934,1852,125174,0
2,경상북도,포항시남구,196727,90434,20195,68137,88332,2102,106293,0


In [177]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_gyeongbuk_21 = g8_gyeongbuk_21[selected_columns].copy()

g8_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상북도,합계,2268707,1194595,904675,255775,0,1160450,34145,1074112
1,경상북도,포항시북구,230960,105786,80784,23150,0,103934,1852,125174
2,경상북도,포항시남구,196727,90434,68137,20195,0,88332,2102,106293


In [178]:
g8_gyeongbuk_21.to_csv("temp2_1_governor_gyeongbuk_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [179]:
g8_gyeongbuk_22 = rename_columns_by_index(g8_gyeongbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_임미애' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_이철우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [180]:
g8_gyeongbuk_22 = add_zero_column(g8_gyeongbuk_22, '그외정당')
g8_gyeongbuk_22 = add_zero_column(g8_gyeongbuk_22, '무소속')

In [181]:
g8_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상북도,합계,2268707,1194595,255775,904675,1160450,34145,1074112,0,0
1,경상북도,포항시북구,230960,105786,23150,80784,103934,1852,125174,0,0
2,경상북도,포항시남구,196727,90434,20195,68137,88332,2102,106293,0,0


In [182]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_gyeongbuk_22 = g8_gyeongbuk_22[selected_columns].copy()

g8_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상북도,합계,2268707,1194595,904675,255775,0,0,1160450,34145,1074112
1,경상북도,포항시북구,230960,105786,80784,23150,0,0,103934,1852,125174
2,경상북도,포항시남구,196727,90434,68137,20195,0,0,88332,2102,106293


In [183]:
g8_gyeongbuk_22.to_csv("temp2_2_governor_gyeongbuk_8.csv", index=False, encoding="utf-8-sig")

## Gyeongnam

In [184]:
g8_gyeongnam.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_양문석,득표수_2_국민의힘_박완수,득표수_3_정의당_여영국,득표수_4_통일한국당_최진석,득표수_계,무효투표수,기권수
0,경상남도,합계,2804287,1497400,431569,963473,58933,12353,1466328,31072,1306887
1,경상남도,창원시의창구,185184,94824,23944,64989,4366,370,93669,1155,90360
2,경상남도,창원시성산구,212811,112825,29902,67977,13523,412,111814,1011,99986
3,경상남도,창원시마산합포구,156850,83924,19970,59943,2417,330,82660,1264,72926
4,경상남도,창원시마산회원구,159983,85597,22063,59376,2601,324,84364,1233,74386


In [185]:
show_column_info(g8_gyeongnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_양문석
5: 득표수_2_국민의힘_박완수
6: 득표수_3_정의당_여영국
7: 득표수_4_통일한국당_최진석
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 23


### v2.1

In [186]:
g8_gyeongnam_21 = rename_columns_by_index(g8_gyeongnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_양문석' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_박완수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [187]:
g8_gyeongnam_21 = add_sum_column(g8_gyeongnam_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_정의당_여영국
7: 득표수_4_통일한국당_최진석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [188]:
g8_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_여영국,득표수_4_통일한국당_최진석,득표수_계,무효투표수,기권수,그외정당
0,경상남도,합계,2804287,1497400,431569,963473,58933,12353,1466328,31072,1306887,71286
1,경상남도,창원시의창구,185184,94824,23944,64989,4366,370,93669,1155,90360,4736
2,경상남도,창원시성산구,212811,112825,29902,67977,13523,412,111814,1011,99986,13935


In [189]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_gyeongnam_21 = g8_gyeongnam_21[selected_columns].copy()

g8_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상남도,합계,2804287,1497400,963473,431569,71286,1466328,31072,1306887
1,경상남도,창원시의창구,185184,94824,64989,23944,4736,93669,1155,90360
2,경상남도,창원시성산구,212811,112825,67977,29902,13935,111814,1011,99986


In [190]:
g8_gyeongnam_21.to_csv("temp2_1_governor_gyeongnam_8.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [191]:
g8_gyeongnam_22 = rename_columns_by_index(g8_gyeongnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_양문석' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_박완수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [192]:
g8_gyeongnam_22 = add_sum_column(g8_gyeongnam_22, 6, 7, '그외정당')
g8_gyeongnam_22 = add_zero_column(g8_gyeongnam_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_정의당_여영국
7: 득표수_4_통일한국당_최진석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [193]:
g8_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_정의당_여영국,득표수_4_통일한국당_최진석,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상남도,합계,2804287,1497400,431569,963473,58933,12353,1466328,31072,1306887,71286,0
1,경상남도,창원시의창구,185184,94824,23944,64989,4366,370,93669,1155,90360,4736,0
2,경상남도,창원시성산구,212811,112825,29902,67977,13523,412,111814,1011,99986,13935,0


In [194]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_gyeongnam_22 = g8_gyeongnam_22[selected_columns].copy()

g8_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상남도,합계,2804287,1497400,963473,431569,71286,0,1466328,31072,1306887
1,경상남도,창원시의창구,185184,94824,64989,23944,4736,0,93669,1155,90360
2,경상남도,창원시성산구,212811,112825,67977,29902,13935,0,111814,1011,99986


In [195]:
g8_gyeongnam_22.to_csv("temp2_2_governor_gyeongnam_8.csv", index=False, encoding="utf-8-sig")

## Jeju

In [196]:
g8_jeju.head()

,시도,구시군,선거인수,투표수,득표수_1_더불어민주당_오영훈,득표수_2_국민의힘_허향진,득표수_4_녹색당_부순정,득표수_5_무소속_박찬식,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,565084,300139,163116,116786,5750,10138,295790,4349,264945
1,제주특별자치도,제주시,409110,216419,118762,83539,4389,6878,213568,2851,192691
2,제주특별자치도,서귀포시,155974,83720,44354,33247,1361,3260,82222,1498,72254


In [197]:
show_column_info(g8_jeju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_더불어민주당_오영훈
5: 득표수_2_국민의힘_허향진
6: 득표수_4_녹색당_부순정
7: 득표수_5_무소속_박찬식
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 3


### v2.1

In [198]:
g8_jeju_21 = rename_columns_by_index(g8_jeju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_오영훈' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_허향진' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [199]:
g8_jeju_21 = add_sum_column(g8_jeju_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_녹색당_부순정
7: 득표수_5_무소속_박찬식

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [200]:
g8_jeju_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_녹색당_부순정,득표수_5_무소속_박찬식,득표수_계,무효투표수,기권수,그외정당
0,제주특별자치도,합계,565084,300139,163116,116786,5750,10138,295790,4349,264945,15888
1,제주특별자치도,제주시,409110,216419,118762,83539,4389,6878,213568,2851,192691,11267
2,제주특별자치도,서귀포시,155974,83720,44354,33247,1361,3260,82222,1498,72254,4621


In [201]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g8_jeju_21 = g8_jeju_21[selected_columns].copy()

g8_jeju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,565084,300139,116786,163116,15888,295790,4349,264945
1,제주특별자치도,제주시,409110,216419,83539,118762,11267,213568,2851,192691
2,제주특별자치도,서귀포시,155974,83720,33247,44354,4621,82222,1498,72254


In [202]:
g8_jeju_21.to_csv("temp2_1_governor_jeju_8.csv", index=False, encoding="utf-8-sig")

### v2.2

In [203]:
g8_jeju_22 = rename_columns_by_index(g8_jeju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_더불어민주당_오영훈' -> '진보정당'
인덱스 5: '득표수_2_국민의힘_허향진' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [204]:
g8_jeju_22 = add_sum_column(g8_jeju_22, 6, 6, '그외정당')
g8_jeju_22 = add_sum_column(g8_jeju_22, 7, 7, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_녹색당_부순정

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12

=== 합산할 열들 (인덱스 7~7) ===
7: 득표수_5_무소속_박찬식

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 13


In [205]:
g8_jeju_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_녹색당_부순정,득표수_5_무소속_박찬식,득표수_계,무효투표수,기권수,그외정당,무소속
0,제주특별자치도,합계,565084,300139,163116,116786,5750,10138,295790,4349,264945,5750,10138
1,제주특별자치도,제주시,409110,216419,118762,83539,4389,6878,213568,2851,192691,4389,6878
2,제주특별자치도,서귀포시,155974,83720,44354,33247,1361,3260,82222,1498,72254,1361,3260


In [206]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g8_jeju_22 = g8_jeju_22[selected_columns].copy()

g8_jeju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,565084,300139,116786,163116,5750,10138,295790,4349,264945
1,제주특별자치도,제주시,409110,216419,83539,118762,4389,6878,213568,2851,192691
2,제주특별자치도,서귀포시,155974,83720,33247,44354,1361,3260,82222,1498,72254


In [207]:
g8_jeju_22.to_csv("temp2_2_governor_jeju_8.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [208]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp2_1_governor_jeonnam_8.csv
Added: temp2_2_governor_sejong_8.csv
Added: temp2_2_governor_jeonbuk_8.csv
Added: temp2_1_governor_daejeon_8.csv
Added: temp2_2_governor_daejeon_8.csv
Added: temp2_1_governor_jeonbuk_8.csv
Added: temp2_2_governor_busan_8.csv
Added: temp2_1_governor_gyeonggi_8.csv
Added: temp2_2_governor_chungnam_8.csv
Added: temp2_2_governor_seoul_8.csv
Added: temp2_1_governor_gyeongbuk_8.csv
Added: temp2_1_governor_chungnam_8.csv
Added: temp2_2_governor_ulsan_8.csv
Added: temp2_1_governor_gwangju_8.csv
Added: temp2_1_governor_jeju_8.csv
Added: temp2_2_governor_incheon_8.csv
Added: temp2_2_governor_gyeongnam_8.csv
Added: temp2_1_governor_incheon_8.csv
Added: temp2_2_governor_chungbuk_8.csv
Added: temp2_1_governor_seoul_8.csv
Added: temp2_2_governor_jeonnam_8.csv
Added: temp2_1_governor_gyeongnam_8.csv
Added: temp2_2_governor_gangwon_8.csv
Added: temp2_1_governor_daegu_8.csv
Added: temp2_2_governor_gyeongbuk_8.csv
Added: temp2_1_governor_ulsan_8.csv
Added: temp2_2_g